In [1]:
!pip install gensim nltk scikit-learn tensorflow seaborn


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("alldata_1_for_kaggle.csv", encoding='latin-1', on_bad_lines='skip')
df.head()

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...


In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

df['clean_text'] = df['a'].apply(clean_text)

In [6]:
print(f"Number of unique values in column 'a': {df['a'].nunique()}")
print("\nTop 20 most frequent values in column 'a':")
display(df['a'].value_counts().head(20))

Number of unique values in column 'a': 996

Top 20 most frequent values in column 'a':


a
the periparturient period is one of the most challenging periods in dairy cows and encompasses the  a0wk prior to and  a0wk after parturition  the  nutrient  requirements  of  dairy  cows  change greatly during this time largely due to the exponential growth of the gravid uterus and fetus followed by the demands of lactation nrc  inflammation oxidative stress and adipose tissue mobilization  lead  to  a  reduction  in  dry  matter  intake  dmi during the periparturient period this reduction in dmi leads to a negative nutrient balance nnb with a shortfall in the nutrient availability  for  the  cow  and  fetus  ingvartsen  and  andersen   additionally  this  reduction  in  dmi  also  increases  the risk of metabolic ketosis fatty liver milk fever and immunerelated disorders the risk of these diseases poor reproduction and low efficiency is greatly impacted by the degree and length of time during which these systems metabolism and immune response remain out of balance loor et a0al 2013a much of the research  over  the  last  decade  have  examined  these  biological interactions to identify the mechanisms behind the metabolic physiologic  and  immune  adaptations  that  occur  during  the periparturient period loor et a0al 2013a 2013b roche et a0al  bradford et a0al it  is  now  known  that  nutrients  such  as  amino  acids  aa serve  functional  roles  outside  of  their  use  as  building  blocks for  proteins  and  have  immunomodulatory  properties  and interact through common biochemical pathways eg 1carbon metabolism  figure a0   this  concept  has  been  well  explored in  nonruminant  species  li  et a0 al    sikalidis    with nutritional  management  during  the  periparturient  period continuing  to  be  an  active  area  of  research  it  is  important  to develop a system understanding the potential immunometabolic role  that  dietary  aa  may  play  during  this  period  thus  the objective of this review is to provide an overview of physiological adaptations  during  the  periparturient  period  the  immune system and methods to assess immune function and oxidative stress this will be followed by a more specific discussion of the immunometabolic roles of specific aa and their potential effects in  dairy  cow  during  the  periparturient  period  the  potential effects  of  enhanced  aa  supply  during  the  preweaning  period will also be discussed brieflybiological adaptations in the transition cowa brief overviewduring  the  nnb  associated  with  the  periparturient  period biological  mechanisms  coordinate  the  mobilization  of  body reserves  in  order  to  support  fetal  growth  and  milk  production bauman  and  currie   insulin concentrations  are  reduced  and  the  response  of  hormonesensitive  lipase  in  adipose  tissue  eg  low  insulin  high growth  hormone  and  catecholamines  or  high  glucocorticoid concentrations  is  greater  to  facilitate  lipid  mobilization this  periparturient  period  is  also  characterized  by  a  state  of inflammation encompassing an increase in hepatic production of  positive  acutephase  proteins  app  such  as  haptoglobin and serum amyloid a a0saa and a decrease in the production ingvartsen    the authors  published by oxford university press on behalf of the american society of animal sciencethis is an open access  distributed under the terms of the creative commons attribution noncommercial license httpcreativecommonslicensesbync40 which permits noncommercial reuse distribution and reproduction in any medium provided the original work is properly cited for commercial reuse please contact  spermissionsoupcoms175 0cs176    of animal science  vol  no suppl abbreviationsaa akt app asct2 bcaa bcat bcka bhmt bmec bsa cbs csad cth dmi gator1 gcl gclc gpx gsh gsr gss il inos kegg klh lps mpo mtor mtorc1 mtr nfe2l2 nfÎºb nnb nos pc pmn pmnl pmtor rns rom ros rpm saa sahh sam samtor sell shmt slc1a1 slc1a5 sod stat tag th tlr tnfÎ± vldl amino acidsprotein kinase b

In [7]:
from sklearn.preprocessing import LabelEncoder

X = df['clean_text']
y = df['0']

# Encoding target labels to numerical values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Using the encoded labels for train_test_split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [9]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()


In [10]:
X_train_tfidf = X_train_tfidf.reshape(X_train_tfidf.shape[0], 1, X_train_tfidf.shape[1])
X_test_tfidf = X_test_tfidf.reshape(X_test_tfidf.shape[0], 1, X_test_tfidf.shape[1])


In [11]:
num_classes = len(le.classes_)

model_tfidf = Sequential([
    SimpleRNN(64, input_shape=(1, X_train_tfidf.shape[2])),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model_tfidf.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_tfidf.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │       324,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 324,355 (1.24 MB)

 Trainable params: 324,355 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history_tfidf = model_tfidf.fit(
    X_train_tfidf, y_train_encoded,
    epochs=50,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6291 - loss: 0.9348 - val_accuracy: 0.8812 - val_loss: 0.5103
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9262 - loss: 0.3974 - val_accuracy: 0.9076 - val_loss: 0.2721
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9400 - loss: 0.2040 - val_accuracy: 0.9092 - val_loss: 0.1839
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9484 - loss: 0.1402 - val_accuracy: 0.9224 - val_loss: 0.1467
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9534 - loss: 0.1126 - val_accuracy: 0.9241 - val_loss: 0.1267
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9574 - loss: 0.0991 - val_accuracy: 0.9406 - val_loss: 0.1198
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9588 - loss: 0.0926 - val_accuracy: 0.9538 - val_loss: 0.1069
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9656 - loss: 0.0801 - val_accuracy: 0.9356 - v

In [13]:
y_pred_probs = model_tfidf.predict(X_test_tfidf)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_tfidf = accuracy_score(y_test_encoded, y_pred)
f1_tfidf = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_tfidf)
print("F1 Score:", f1_tfidf)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Accuracy: 0.9775429326287979
F1 Score: 0.9775508181336324
                precision    recall  f1-score   support

  Colon_Cancer       0.94      1.00      0.97       516
   Lung_Cancer       1.00      1.00      1.00       436
Thyroid_Cancer       1.00      0.94      0.97       562

      accuracy                           0.98      1514
     macro avg       0.98      0.98      0.98      1514
  weighted avg       0.98      0.98      0.98      1514



In [14]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


In [15]:
sentences = [text.split() for text in X_train]

w2v_skipgram = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1  # Skip-gram
)


In [16]:
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))

for word, i in word_index.items():
    if word in w2v_skipgram.wv:
        embedding_matrix[i] = w2v_skipgram.wv[word]


In [17]:
model_skipgram = Sequential([
    Embedding(len(word_index)+1, embedding_dim, weights=[embedding_matrix],
              input_length=max_len, trainable=False),
    SimpleRNN(64),
    Dropout(0.3),
    Dense(num_classes, activation='softmax') # Changed for multi-class classification
])

model_skipgram.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Changed loss for multi-class integer labels
model_skipgram.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │    17,687,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,687,800 (67.47 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 17,687,800 (67.47 MB)

In [18]:
history_skip = model_skipgram.fit(
    X_train_pad, y_train_encoded,
    epochs=50,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.4146 - loss: 1.1500 - val_accuracy: 0.6271 - val_loss: 0.7977
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.6493 - loss: 0.7542 - val_accuracy: 0.7805 - val_loss: 0.6100
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.7657 - loss: 0.5631 - val_accuracy: 0.8086 - val_loss: 0.4845
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.8433 - loss: 0.4125 - val_accuracy: 0.8581 - val_loss: 0.3520
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9081 - loss: 0.2626 - val_accuracy: 0.9158 - val_loss: 0.2259
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9410 - loss: 0.1792 - val_accuracy: 0.9373 - val_loss: 0.1707
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9485 - loss: 0.1495 - val_accuracy: 0.9406 - val_loss: 0.1504
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9608 - loss: 0.1162 - val_accuracy: 0.9439 - v

In [19]:
y_pred_probs = model_skipgram.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_skip = accuracy_score(y_test_encoded, y_pred)
f1_skip = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_skip)
print("F1 Score:", f1_skip)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Accuracy: 0.987450462351387
F1 Score: 0.9874550995390161
                precision    recall  f1-score   support

  Colon_Cancer       0.97      0.99      0.98       516
   Lung_Cancer       1.00      1.00      1.00       436
Thyroid_Cancer       0.99      0.97      0.98       562

      accuracy                           0.99      1514
     macro avg       0.99      0.99      0.99      1514
  weighted avg       0.99      0.99      0.99      1514



In [20]:
w2v_cbow = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=0  # CBOW
)


In [21]:
embedding_matrix_cbow = np.zeros((len(word_index)+1, embedding_dim))

for word, i in word_index.items():
    if word in w2v_cbow.wv:
        embedding_matrix_cbow[i] = w2v_cbow.wv[word]


In [22]:
from tensorflow.keras.layers import LSTM

model_cbow = Sequential([
    Embedding(len(word_index)+1, embedding_dim, weights=[embedding_matrix_cbow],
              input_length=max_len, trainable=False),
    SimpleRNN(64), # Reverted to SimpleRNN with original units
    Dropout(0.3), # Reverted dropout rate
    Dense(num_classes, activation='softmax')
])

model_cbow.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cbow.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │    17,687,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,687,800 (67.47 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 17,687,800 (67.47 MB)

In [23]:
history_cbow = model_cbow.fit(
    X_train_pad, y_train_encoded,
    epochs=50,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.4230 - loss: 1.2744 - val_accuracy: 0.6584 - val_loss: 0.7505
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.6288 - loss: 0.7967 - val_accuracy: 0.7673 - val_loss: 0.5918
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.7380 - loss: 0.6132 - val_accuracy: 0.8333 - val_loss: 0.4771
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.8008 - loss: 0.5072 - val_accuracy: 0.8762 - val_loss: 0.3833
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.8362 - loss: 0.4255 - val_accuracy: 0.8911 - val_loss: 0.3151
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8831 - loss: 0.3422 - val_accuracy: 0.9191 - val_loss: 0.2508
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9020 - loss: 0.2861 - val_accuracy: 0.9175 - val_loss: 0.2113
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9148 - loss: 0.2383 - val_accuracy: 0.9274 - v

In [24]:
y_pred_probs = model_cbow.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_cbow = accuracy_score(y_test_encoded, y_pred)
f1_cbow = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_cbow)
print("F1 Score:", f1_cbow)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Accuracy: 0.9841479524438573
F1 Score: 0.9841550499192745
                precision    recall  f1-score   support

  Colon_Cancer       0.96      1.00      0.98       516
   Lung_Cancer       1.00      1.00      1.00       436
Thyroid_Cancer       1.00      0.96      0.98       562

      accuracy                           0.98      1514
     macro avg       0.99      0.99      0.99      1514
  weighted avg       0.98      0.98      0.98      1514



In [25]:
results = pd.DataFrame({
    "Model": ["RNN + TFIDF", "RNN + Skipgram", "RNN + CBOW"],
    "Accuracy": [acc_tfidf, acc_skip, acc_cbow],
    "F1 Score": [f1_tfidf, f1_skip, f1_cbow]
})

results

,Model,Accuracy,F1 Score
0,RNN + TFIDF,0.977543,0.977551
1,RNN + Skipgram,0.987450,0.987455
2,RNN + CBOW,0.984148,0.984155
